Import Packages

In [1]:
import orbit
from satellite import Satellite
import numpy as np
from scipy.integrate import ode, DOP853
import matplotlib.pyplot as plt
from dynamics import equation_of_motion

Initialize Orbit using one of the following methods:
- Classical Elements
- State Vector

If necessary, implement the conversion to State Vector for the following:
- Delaunay Elements
- Equinoctial Elements

These functionalities should be added from the Orbit abstract class in orbit.py

In [2]:
# Input variables into Orbit class from classical 
# orbital elements or position and velocity
a  = 6378 + 100
e  = 0.01
i  = 97
w  = 30
RA = 45
TA = 40

orbit0 = orbit.OrbitElements(a,e,i,w,RA,TA)
R, mu = orbit0.get_central_body() # Get central body according to orbit
orbit0.activate_perturbation(["Drag"])  # Select perturbations

sat0 = Satellite()                # Satellite in orbit

Initial condition
Position: [2075.13813018 1034.07135414 5995.42297169]
Velocity: [-5.00310908 -5.47723189  2.73043656]

State Vector: [ 2.07513813e+03  1.03407135e+03  5.99542297e+03 -5.00310908e+00
 -5.47723189e+00  2.73043656e+00]

Using atmospheric drag.


Simulation set up

In [3]:
# Define simulation time
t0        = 0
time_end  = 24*3600
step      = 1
times     = np.arange(0,time_end,step=step)

Linear Control

In [4]:
# Tune control law used
Kp = 0
Ki = 0
Kd = 0

Simulation

In [5]:
# Simulate orbit
solver  = ode(equation_of_motion)
solver.set_integrator('DOP853')
solver.set_f_params(orbit0,sat0)
solver.set_initial_value(orbit0.initial_state,t0)

X=[];Y=[];Z=[];Vx=[];Vy=[];Vz=[]

for time in times:
    solver.integrate(time)
    X.append(solver.y[0])
    Y.append(solver.y[1])
    Z.append(solver.y[2])
    Vx.append(solver.y[3])
    Vy.append(solver.y[4])
    Vz.append(solver.y[5])




/home/msb/Codes/PY/orbital-mechanics/env/lib/python3.10/site-packages/scipy/integrate/_ode.py:431: UserWarning: dop853: step size becomes too small
  self._y, self.t = mth(self.f, self.jac or (lambda: None),


In [ ]:
# Plot graphics and return coordinates in ECI frame
graph = plt.figure(1)
plt.subplot(311)
plt.plot(times/3600,X,label='X')
plt.ylim([-7_000,7_000])
plt.legend()

plt.subplot(312)
plt.plot(times/3600,Y,label='Y')
plt.ylim([-7_000,7_000])
plt.ylabel("Position (km)")
plt.legend()

plt.subplot(313)
plt.plot(times/3600,Z,label='Z')
plt.ylim([-7_000,7_000])
plt.xlabel("Time (h)")
plt.legend()

altitude = plt.figure(2)
alt = [np.sqrt(X[i]**2+Y[i]**2+Z[i]**2) for i in range(len(times))]
plt.plot(times/3600,alt,label='Altitude')
plt.legend()

tri = plt.figure(3)
ax = tri.add_subplot(111,projection='3d')
ax.plot(X,Y,Z, label='3D')
ax.set_xlim((-7000,7000))
ax.set_ylim((-7000,7000))
ax.set_zlim((-7000,7000))
ax.legend()

plt.show()
